In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
from typing import List, Text

import tensorflow_model_analysis as tfma
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import Pusher
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.components.base import executor_spec
from tfx.dsl.components.common import resolver
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import example_gen_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.types import Channel

from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing

In [2]:
_pipeline_name = 'movies_genre'
base_dir = os.getcwd()
_project_root = os.path.join("data/")
_data_root = os.path.join(_project_root)
# Python module file to inject customized logic into the TFX components. The
# Transform and Trainer both require user-defined functions to run successfully.
_module_file = os.path.join(base_dir, 'movies_utils_native_keras.py')
# Path which can be listened to by the model server.  Pusher will output the
# trained model here.
_serving_model_dir = os.path.join(base_dir, 'serving_model', _pipeline_name)

# Directory and data locations. This example assumes all of the
# example code and metadata library is relative to $HOME, but you can store
# these files anywhere on your local filesystem.
_tfx_root = os.path.join(os.environ['HOME'], 'tfx')
_pipeline_root = os.path.join(_tfx_root, 'pipelines', _pipeline_name)
# Sqlite ML-metadata db path.
_metadata_path = os.path.join(_tfx_root, 'metadata', _pipeline_name, 'metadata.db')

context = InteractiveContext()

pipeline_name = _pipeline_name
pipeline_root = _pipeline_root
data_root = _data_root
module_file = _module_file
serving_model_dir = _serving_model_dir
metadata_path = _metadata_path

output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=9),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1)
    ]))

# Brings data in to the pipline

data_dir = os.path.join(base_dir, "data")

print(data_dir)
example_gen = CsvExampleGen(input_base=data_dir, output_config=output)
context.run(example_gen)
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

/home/imad/imworkspace/movie_nlp/tfx/data


["train", "eval"] /tmp/tfx-interactive-2021-06-13T17_47_45.623478-5pcsf16l/CsvExampleGen/examples/1


In [3]:
# Computes statistics over data for visualization and example validation.
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)
context.show(statistics_gen.outputs['statistics'])

In [4]:
# Generates schema based on statistics files.
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 9
        uri: "/tmp/tfx-interactive-2021-06-13T17_47_45.623478-5pcsf16l/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 9
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [5]:
# Performs anomaly detection based on statistics and data schema.
example_validator = ExampleValidator(statistics=statistics_gen.outputs['statistics'], schema=schema_gen.outputs['schema'])
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 4
        type_id: 11
        uri: "/tmp/tfx-interactive-2021-06-13T17_47_45.623478-5pcsf16l/ExampleValidator/anomalies/4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 11
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [6]:
# Performs transformations and feature engineering in training and serving.
print(module_file)
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=module_file)
context.run(transform)

/home/imad/imworkspace/movie_nlp/tfx/movies_utils_native_keras.py
ERROR:absl:udf_utils.get_fn {'module_file': None, 'module_path': 'movies_utils_native_keras@/tmp/tfx-interactive-2021-06-13T17_47_45.623478-5pcsf16l/_wheels/tfx_user_code_Transform-0.0+52ffe47dc9b8fa4d6cd544b6056c64dd0d541298917ae0a675e09d126ed55f04-py3-none-any.whl', 'preprocessing_fn': None} 'preprocessing_fn'
Tensor("inputs_copy:0", shape=(None, 1), dtype=int64)
Instructions for updating:
Use ref() instead.
Instructions for updating:
Use ref() instead.
Tensor("inputs_copy:0", shape=(None, 1), dtype=int64)
Tensor("inputs_copy:0", shape=(None, 1), dtype=int64)
Tensor("inputs_copy:0", shape=(None, 1), dtype=int64)
Tensor("inputs_copy:0", shape=(None, 1), dtype=int64)
Tensor("inputs_copy:0", shape=(None, 1), dtype=int64)
Tensor("PlaceholderWithDefault:0", shape=(None, 1), dtype=int64)
Tensor("inputs_copy:0", shape=(None, 1), dtype=int64)
INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2021-06-13T17_47_45.623478-5p

ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 5
        type_id: 13
        uri: "/tmp/tfx-interactive-2021-06-13T17_47_45.623478-5pcsf16l/Transform/transform_graph/5"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 13
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 6
        type_id: 5
        uri: "/tmp/tfx-interactive-2021-06-13T17_47_45.623478-5pcsf16l/Transform/transformed_examples/5"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 7
        type_id: 14
        uri: "/tmp/tfx-interactive-2021-06-13T17_47_45.623478-5pcsf16l/Transform/updated_analyzer_cache/5"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [7]:
# Uses user-provided Python function that trains a model.
trainer = Trainer(
    module_file=module_file,
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=500),
    eval_args=trainer_pb2.EvalArgs(num_steps=200))
context.run(trainer)

ERROR:absl:udf_utils.get_fn {'train_args': '{\n  "num_steps": 500\n}', 'eval_args': '{\n  "num_steps": 200\n}', 'module_file': None, 'run_fn': None, 'trainer_fn': None, 'custom_config': 'null', 'module_path': 'movies_utils_native_keras@/tmp/tfx-interactive-2021-06-13T17_47_45.623478-5pcsf16l/_wheels/tfx_user_code_Trainer-0.0+52ffe47dc9b8fa4d6cd544b6056c64dd0d541298917ae0a675e09d126ed55f04-py3-none-any.whl'} 'run_fn'
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
500/500 [==============================] - 83s 157ms/step - loss: 2.7595 - accuracy: 0.1922 - val_loss: 2.5209 - val_accuracy: 0.1840
INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2021-06-13T17_47_45.623478-5pcsf16l/Trainer/model/6/Format-Serving/assets
INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2021-06-13T17_47_45.623478-5pcsf16l/Trainer/model/6/Form

ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 8
        type_id: 16
        uri: "/tmp/tfx-interactive-2021-06-13T17_47_45.623478-5pcsf16l/Trainer/model/6"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 9
        type_id: 17
        uri: "/tmp/tfx-interactive-2021-06-13T17_47_45.623478-5pcsf16l/Trainer/model_run/6"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 17
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [8]:
# Get the latest blessed model for model validation.
model_resolver = resolver.Resolver(
    instance_name=Channel(type=ModelBlessing),
    strategy_class=latest_blessed_model_resolver.LatestBlessedModelResolver,
    model=Channel(type=Model),
    model_blessing=Channel(type=ModelBlessing))

In [9]:
# Uses TFMA to compute evaluation statistics over features of a model and
# perform quality validation of a candidate model (compared to a baseline).
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='label')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(
                class_name='Accuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        # Increase this threshold when training on complete
                        # dataset.
                        lower_bound={'value': 0.01}),
                    # Change threshold will be ignored if there is no
                    # baseline model resolved from MLMD (first run).
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value': -1e-2})))
        ])
    ])

In [11]:
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)

In [12]:
# Checks whether the model passed the validation steps and pushes the model
# to a file destination if check passed.
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=serving_model_dir)))
context.run(pusher)
print(pusher.outputs)



{'pushed_model': Channel(
    type_name: PushedModel
    artifacts: [Artifact(artifact: id: 10
type_id: 19
uri: "/tmp/tfx-interactive-2021-06-13T17_47_45.623478-5pcsf16l/Pusher/pushed_model/7"
custom_properties {
  key: "name"
  value {
    string_value: "pushed_model"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "Pusher"
  }
}
custom_properties {
  key: "pushed"
  value {
    int_value: 1
  }
}
custom_properties {
  key: "pushed_destination"
  value {
    string_value: "/home/imad/imworkspace/movie_nlp/tfx/serving_model/movies_genre/1623595844"
  }
}
custom_properties {
  key: "pushed_version"
  value {
    string_value: "1623595844"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "0.30.0"
  }
}
state: LIVE
, artifact_type: id: 19
name: "PushedModel"
)]
    additional_properties: {}
    additional_custom_properties: {}
)}
